In [1]:
from qaravan.tensorQ import *
from qaravan.core import *

In [2]:
# testing if we can prepare GHZ state
gate_list = [H(0), CNOT([1,0], 3), CNOT([2,1], 3)]
circ = Circuit(gate_list, 3)
print(circ)

sv = StatevectorSim(circ)
final_state = sv.run()
print(sv)

H gate on site(s) [0] 
bottom heavy CNOT2 gate on site(s) [0, 1] 
bottom heavy CNOT2 gate on site(s) [1, 2]


100%|██████████| 3/3 [00:00<00:00, 1498.14it/s]

0.7071|000⟩ + 0.7071|111⟩


In [3]:
# testing local expectations
local_ops = [pauli_Z, pauli_Z, pauli_Z]
sv.local_expectation(local_ops) # should be zero

array(0.)

In [4]:
# testing local expectations again
local_ops = [pauli_X, pauli_X, pauli_X]
sv.local_expectation(local_ops) # should be 1

array(1.)

In [5]:
# testing environments 
n = 4
sv = random_sv(n)

gate_list = [H(0), CNOT([1,0], n)]#, CNOT([2,1], n), CNOT([3,2], n)]
circ = Circuit(gate_list, n)
sim = StatevectorSim(circ)
final_state = sim.run(progress_bar=False).reshape(2**n)
overlap_a = sv.conj().T @ final_state

for gate_idx in range(len(circ)):
    env, mat = sv_environment(circ, sv, gate_idx)
    overlap_b = np.trace(env @ mat)
    print(np.allclose(overlap_a, overlap_b))

True
True


In [6]:
# testing environments from cached states 

def test_fast_environment():
    target_sv = random_sv(4)
    circ = two_local_circ([(0,1), (2,3), (1,2), (0,1), (2,3), (1,2)])

    for gate_idx in range(len(circ)):
        env_slow, _ = sv_environment(circ, target_sv, gate_idx)
        pre_states, post_states = cache_states(circ, target_sv)
        env_fast = partial_overlap(pre_states[gate_idx], post_states[gate_idx], skip=circ[gate_idx].indices)

        assert np.allclose(env_fast, env_slow), f"Mismatch in env at gate {gate_idx}"

test_fast_environment()

In [7]:
# testing environment_update()

def test_environment_update():
    target_sv = random_sv(4)
    circ = two_local_circ([(0,1), (2,3), (1,2), (0,1), (2,3), (1,2)])
    pre_states, post_states = cache_states(circ, target_sv)

    simple_costs, update_costs, cache_costs = [], [], []
    for idx in range(len(circ)-1):
        update_costs.append(environment_update(circ, idx, pre_states, post_states, direction='right'))

        sim = StatevectorSim(circ)
        ansatz = sim.run(progress_bar=False).reshape(2**n)
        simple_costs.append(1 - np.abs(target_sv.conj().T @ ansatz))

        pre_state = pre_states[idx+1]
        post_state = post_states[idx+1]
        env = partial_overlap(pre_state, post_state, skip=circ[idx+1].indices)
        cache_costs.append(1 - np.abs(np.trace(env @ circ[idx+1].matrix)))

    for idx in reversed(range(1, len(circ))):
        update_costs.append(environment_update(circ, idx, pre_states, post_states, direction='left'))
    
        sim = StatevectorSim(circ)
        ansatz = sim.run(progress_bar=False).reshape(2**n)
        simple_costs.append(1 - np.abs(target_sv.conj().T @ ansatz))

        pre_state = pre_states[idx-1]
        post_state = post_states[idx-1]
        env = partial_overlap(pre_state, post_state, skip=circ[idx-1].indices)
        cache_costs.append(1 - np.abs(np.trace(env @ circ[idx-1].matrix)))

    assert np.allclose(simple_costs, update_costs), "Mismatch in update costs"
    assert np.allclose(simple_costs, cache_costs), "Mismatch in cache costs"

test_environment_update()

In [8]:
# testing environment based state prep
skeleton = [(0,1), (2,3), (1,2), (0,1), (2,3)]
target_sv = random_sv(4)

context = RunContext(
    progress_interval=15,
    max_iter=50,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=40,
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::40]

Step 0 at time 2025-04-16T11:55:29: cost = 0.0029558548394018613
[Checkpoint saved at step 0]
Step 15 at time 2025-04-16T11:55:30: cost = 1.0152272896357672e-05
Step 30 at time 2025-04-16T11:55:30: cost = 2.4790932495744045e-07
[Checkpoint saved at step 40]
Step 45 at time 2025-04-16T11:55:30: cost = 5.382723933244904e-09
Max iterations reached with cost 1.9308893373093383e-09


(40,
 [np.float64(0.7815720512402541),
  np.float64(0.00011374839190969599),
  np.float64(3.968873941528894e-05),
  np.float64(1.2840492270349557e-05),
  np.float64(3.8790906119468005e-06),
  np.float64(1.1253452445458834e-06),
  np.float64(3.194254728500212e-07),
  np.float64(8.963953890273046e-08),
  np.float64(2.500649320946735e-08)])

In [9]:
# resuming from previous run
context = RunContext(
    progress_interval=15,
    max_iter=100,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=40,
    resume=True
    )
    
circ, cost_list = environment_state_prep(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::40]

[Resuming from checkpoint: step 40]
[Checkpoint saved at step 40]
Step 45 at time 2025-04-16T11:55:30: cost = 4.1660095373430295e-09
Step 60 at time 2025-04-16T11:55:30: cost = 8.890665981198254e-11
Step 75 at time 2025-04-16T11:55:30: cost = 1.892930256985892e-12
[Checkpoint saved at step 80]
Step 90 at time 2025-04-16T11:55:30: cost = 4.107825191113079e-14
Max iterations reached with cost 4.218847493575595e-15


(80,
 [np.float64(0.7815720512402541),
  np.float64(0.00011374839190969599),
  np.float64(3.968873941528894e-05),
  np.float64(1.2840492270349557e-05),
  np.float64(3.8790906119468005e-06),
  np.float64(1.1253452445458834e-06),
  np.float64(3.194254728500212e-07),
  np.float64(8.963953890273046e-08),
  np.float64(2.500649320946735e-08),
  np.float64(6.9543973912900015e-09),
  np.float64(1.9308893373093383e-09),
  np.float64(5.356476373563623e-10),
  np.float64(1.485251921451436e-10),
  np.float64(4.117306495743378e-11),
  np.float64(1.1411982470121984e-11),
  np.float64(3.1624702856447584e-12),
  np.float64(8.772982340587987e-13)])

In [10]:
# testing environment based state prep
skeleton = [(0,1), (2,3), (1,2), (0,1), (2,3)]
target_sv = random_sv(4)

context = RunContext(
    progress_interval=15,
    max_iter=50,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=40,
    )
    
circ, cost_list = environment_state_prep_simple(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::40]

Step 0 at time 2025-04-16T11:55:34: cost = 0.05625031463862151
[Checkpoint saved at step 0]
Step 15 at time 2025-04-16T11:55:34: cost = 0.0002009947594969752
Step 30 at time 2025-04-16T11:55:34: cost = 0.00016107182892766136
[Checkpoint saved at step 40]
Step 45 at time 2025-04-16T11:55:34: cost = 0.00013952555461294036
Max iterations reached with cost 0.0001353476407063292


(40, [np.float64(0.18975155803426774), np.float64(0.0001468346715296498)])

In [11]:
# resuming from previous run
context = RunContext(
    progress_interval=15,
    max_iter=100,
    stop_ratio=1e-8,
    checkpoint_file="test.pickle", 
    checkpoint_interval=40,
    resume=True
    )
    
circ, cost_list = environment_state_prep_simple(target_sv, skeleton=skeleton, context=context)

with open('test.pickle', 'rb') as f: 
    opt_state = pickle.load(f)

opt_state['step'], opt_state['cost_list'][::40]

[Resuming from checkpoint: step 40]
[Checkpoint saved at step 40]
Step 45 at time 2025-04-16T11:56:07: cost = 0.0001384347842899869
Step 60 at time 2025-04-16T11:56:08: cost = 0.00012518828178464947
Step 75 at time 2025-04-16T11:56:08: cost = 0.0001157048444282438
[Checkpoint saved at step 80]
Step 90 at time 2025-04-16T11:56:08: cost = 0.00010815608304137747
Max iterations reached with cost 0.00010410790523807112


(80,
 [np.float64(0.18975155803426774),
  np.float64(0.0001468346715296498),
  np.float64(0.00011408096419907032)])